<a href="https://colab.research.google.com/github/daikichiba9511/study_log/blob/master/pyorch_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***pytorch***

チュートリアル



In [0]:
import torch

In [32]:
#初期化された(5,3)行列をTensor型で宣言
x=torch.Tensor(5,3)
x

tensor([[1.4643e-36, 0.0000e+00, 4.4842e-44],
        [0.0000e+00,        nan, 2.6908e+20],
        [8.5388e-07, 1.6690e+22, 3.3796e+21],
        [1.0500e-08, 8.3560e+20, 4.2726e-05],
        [3.3585e-06, 3.3056e+21, 1.3919e-40]])

In [33]:
#乱数で初期化したものをTensor型で宣言する
y=torch.rand(5,3)
y

tensor([[0.4079, 0.3497, 0.5343],
        [0.5806, 0.2269, 0.6082],
        [0.8427, 0.1700, 0.8972],
        [0.4785, 0.4565, 0.9293],
        [0.1650, 0.8715, 0.1737]])

In [34]:
#サイズの確認
x.size()

torch.Size([5, 3])

In [35]:
x+y

tensor([[4.0785e-01, 3.4967e-01, 5.3425e-01],
        [5.8061e-01,        nan, 2.6908e+20],
        [8.4272e-01, 1.6690e+22, 3.3796e+21],
        [4.7850e-01, 8.3560e+20, 9.2931e-01],
        [1.6503e-01, 3.3056e+21, 1.7369e-01]])

In [36]:
#加算
result=torch.Tensor(5,3) #make empty (5,3) Tensor
torch.add(x,y,out=result)
result

tensor([[4.0785e-01, 3.4967e-01, 5.3425e-01],
        [5.8061e-01,        nan, 2.6908e+20],
        [8.4272e-01, 1.6690e+22, 3.3796e+21],
        [4.7850e-01, 8.3560e+20, 9.2931e-01],
        [1.6503e-01, 3.3056e+21, 1.7369e-01]])

In [37]:
y.add_(x)  #"add_(x)" -> y=y+x

tensor([[4.0785e-01, 3.4967e-01, 5.3425e-01],
        [5.8061e-01,        nan, 2.6908e+20],
        [8.4272e-01, 1.6690e+22, 3.3796e+21],
        [4.7850e-01, 8.3560e+20, 9.2931e-01],
        [1.6503e-01, 3.3056e+21, 1.7369e-01]])

In [0]:
from torch.autograd import Variable

In [39]:
#try to make variable
x=Variable(torch.ones(2,2),requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


In [40]:
#add
y=x+2
print(y)
#mul
z=x*4
z

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


tensor([[4., 4.],
        [4., 4.]], grad_fn=<MulBackward0>)

In [41]:
#演算後に生成されたVariableには属性grad_fnがふよされる
y.grad_fn

In [46]:
z2=y*y*3
out=z2.mean()
print(z2,out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward1>)


# 勾配計算
 
*逆伝播の計算を行う* 

In [0]:
out.backward()

In [48]:
x.grad

tensor([[5.5000, 5.5000],
        [5.5000, 5.5000]])

In [55]:
x=torch.randn(3)
x=Variable(x,requires_grad=True)

y=x*2
while y.data.norm()<1000:
  y=y*2
  
print(y)

tensor([551.6786, 473.2256, 707.1232], grad_fn=<MulBackward0>)


In [51]:
gradients = torch.FloatTensor([0.1,1.0,0.0001])
y.backward(gradients)

print(x.grad)

tensor([1.0240e+02, 1.0240e+03, 1.0240e-01])


In [0]:
#手書き文字を分類するNN
#パッケージのインポート
import torch
from torch.autograd  import Variable
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
  
  def __init__(self):
    super(Net,self).__init__()
    #1 input image channel, 6 output chabbels, 5*5 square convolution
    #kernel
    self.conv1=nn.Conv2d(1,6,5) #(input,channels output )
    self.conv2=nn.Conv2d(6,16,5)
    
    #an affine operattion: y=Wx+b　,full connection
    self.fc1=nn.Linear(16*5*5,120) #(input,output)
    self.fc2=nn.Linear(120,84)
    self.fc3=nn.Linear(84,10)
    
  def forward(self,x): 
    #max pooling over a (2,2) window
    x=F.max_pool2d(F.relu(self.conv1(x)),(2,2))
    #if the size is a square you can only specify a single number
    x=F.max_pool2d(F.relu(self.conv2(x),2))
    x=x.view(-1,self.num_flat_features(x))
    x=F.relu(self.fc1(x))
    x=F.relu(self.fc2(x))
    x=self.fc3(x)
    return x
  
  def num_flat_features(self,x):
    size=x.size()[1:] #all dimensions except the batch dimension
    num_features=1
    for s in size:
      num_features *= s
      return num_features
    

  
    

In [58]:
net=Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)
